In [1]:
import yaml
import pathlib
import random
import pandas as pd



def load_recipe(path):
    """ load recipe at path
    """
    with path.open("r") as f:
        return yaml.load(f, Loader=yaml.FullLoader)

def get_recipes():
    blacklist = [
        "Chokoladetærte med saltkaramel",
        "Peberkager",
        "Coleslaw",
        "Dilddressing",
        'Pitabrød',
        'Knuste Kartofler',
        'Flødekartofler',
        'Blomkålssalat med æbler og mandler',
        'Kartoffelmos',
        'Broccolisalat',
        'Anders Ands kanelsnegle',
        'Hvidkålssalat',
        'Rucolasalat med bagte tomater',
        'Raita',
        'Pirogger med oksekød',
        'Pita brød',
        'Mango lassi',
        'TikTok baked oats',
        'Spinatsalat med feta og granatæble',
        'Spinat Pandekager',
        'Pizzadej',
        'Raw muslibar',
        'Mettes Æblekage',
        'Rugboller',
        'Naan',
        'Bedstemor ands chokoladekage',
        "Verdens bedste burger",
        "Kyllinge Nuggets med Corn Flakes",
        "Calzoneboller",
        "Bananbrød",
        "standard",
    ]

    paths = pathlib.Path("recipes").glob("*.yml")
    recipes = {p.stem:load_recipe(p) for p in paths}
    return {rec["navn"]: {**rec, "path":p} for p,rec in recipes.items() if not rec["navn"] in blacklist}


recipes = get_recipes()


for rec in recipes.values():
    rec["filename"] = rec["path"]
    rec["ingredients"] = list(rec["ingredienser"].keys())
    del rec["ingredienser"]
    del rec["navn"]
    del rec["placering"]
    del rec["path"]
    del rec["antal"]

import json

with open("recipes.json", "w") as f:
    json.dump(recipes, f)
    
    
    
    
import pathlib
from collections import defaultdict
import click

import yaml


def add_ingredient(ingrediens, amount, shopping, config, recipe_name):
        category = config["varer"].get(ingrediens, "unknown")
        priority = config["kategorier"][category]
        shopping[priority, ingrediens][amount["unit"]] += amount["amount"]

        if not "recipes" in shopping[priority, ingrediens]:
            shopping[priority, ingrediens]["recipes"] = []

        shopping[priority, ingrediens]["recipes"].append(recipe_name)

def add_recipe(recipe, amount, shopping, config, silent=False):
    recipe_pth = pathlib.Path.cwd() / f"recipes/{recipe}.yml"
    with recipe_pth.open("r") as f: 
        recipe = yaml.load(f, Loader=yaml.FullLoader)

    multiplier = amount["amount"] if amount["unit"] == "recipe" else amount["amount"]/float(recipe["antal"])

    if multiplier == 1:
        amount_str = ""
    else:
        amount_str = f"(x{multiplier:g})" if multiplier else "(fryser)"

    for ingrediens, amount in recipe["ingredienser"].items():
        amount["amount"] *= multiplier
        if amount["amount"]>0:
            add_ingredient(ingrediens, amount, shopping, config, recipe['navn'])

    for ingrediens, amount in recipe.get("extras", {}).items():
        add_ingredient(ingrediens, amount, shopping, config, recipe['navn'])
    
    if silent:
        return ""

    return f" - {recipe['navn']:35s}  {amount_str} : {recipe['placering']:25s}"


def amount_string(amounts, ingrediens):
    to_str = lambda v: f"{v:.2f}".rstrip('0').rstrip('.')
    a = [f"{to_str(amount)} {unit:<6s}" for unit, amount in amounts.items() if not unit=="recipes"]

    amounts_str = " + ".join(a)

    max_len = 12
    included_in = sorted(set(amounts.get('recipes', [])))

    included_in_more_than_self = len(included_in)>1
    if included_in_more_than_self:
        # eg ingredient == apple : [apple, dish-w-apple, other-dish-w-apple] -> [dish-w-apple, other-dish-w-apple]
        # then don't include self
        included_in = [s for s in included_in if not s == ingrediens]
    
    # [dish-w-apple, other-dish-w-apple] -> [dish-w-app, other-dish]
    included_in = [s[:max_len] for s in included_in]
    
    recs_str = f"{' + '.join(included_in)}"

    return amounts_str, recs_str


def print_shopping(shopping):
    """

    |          |          |       |
    |----------|----------|------:|
    | **agurk**    | 1 stk    | agurk |


    """
    prev_priority = None
    new_table = """

|          |          |       |
|----------|----------|------:|"""
    for (priority, ingrediens), amount in sorted(shopping.items()):
        if not (priority == prev_priority):
            yield new_table
        prev_priority = priority
        amounts_str, recs_str = amount_string(amount, ingrediens)
        max_recs_len = 30
        yield f"| {ingrediens:40s} | {amounts_str:>10s} |  {recs_str[:max_recs_len]:30s}  | "

def write_md_str(menu_d):
    with (pathlib.Path.cwd() / "config.yml").open("r") as f: 
        config = yaml.load(f, Loader=yaml.FullLoader)

    shopping = defaultdict(lambda: defaultdict(float) )
    
    
    s = ""

    s += "# Menu"
    s += "\n"
    for name, recipes in menu_d.items():
        silent = name.lower() == "andet"
        if not silent:
            s += "## " + name
            s += "\n"
        for recipe in recipes:
            if isinstance(recipe, str):
                recipe = {recipe: {"amount": 1, "unit": "recipe"}}

            recipe_name, = recipe.keys()
            amount = recipe[recipe_name]
            try:
                s += add_recipe(recipe_name, amount, shopping, config, silent=silent)
                s += "\n"
            except FileNotFoundError:
                if amount["amount"] > 0:
                    add_ingredient(recipe_name, amount, shopping, config, recipe_name)


    s += "# Shopping"
    s += "\n"

    for s_ in print_shopping(shopping):
        s += s_
        s += "\n"
    
    return s

def save_to_md(s):
    with open("shopping.md", "w") as f:
        f.write(s)

import subprocess
import time
import datetime


def md_to_pdf():
    # Define the command as a list of arguments
    command = [
        "pandoc",
        "shopping.md",
        "-f",
        "gfm",
        "-H",
        "chapter_break.tex",
        "-V",
        "geometry:a4paper",
        "-V",
        "geometry:margin=4cm",
        "-V",
        "mainfont=Montserrat",
        "-V",
        "monofont=DejaVu Sans Mono",
        "--pdf-engine=xelatex",
        "-o",
        "shopping.pdf"
    ]

    # Execute the command
    subprocess.run(command)

def open_pdf():
    # Open the generated PDF file
    subprocess.run(["xdg-open", "shopping.pdf"])    
    


In [2]:
assistant_id = ""
api_key = ""

from openai import OpenAI
client = OpenAI(api_key = api_key)

my_assistant = client.beta.assistants.retrieve(assistant_id)



In [6]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "make a pdf for a 6 day menu please. We need 4 portions for each day"
        }
    ]
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=my_assistant.id
)


start = datetime.datetime.now()

while run.status != 'completed':
    run = client.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id
    )
    if run.status == 'requires_action':
        break

thread_messages = client.beta.threads.messages.list(thread.id)
run.status

'requires_action'

In [7]:
for msg in thread_messages.data[::-1]:
    print(msg.content[0].text.value)


make a pdf for a 6 day menu please. We need 4 portions for each day


In [8]:
args_str = run.required_action.submit_tool_outputs.tool_calls[0].function.arguments
func_str = run.required_action.submit_tool_outputs.tool_calls[0].function.name

In [ ]:
menu_dict = json.loads(args_str)
menu_dict

In [ ]:

menu_item_template = """
{title}:
    - {filename}:
        amount: {portions}
        unit: plates
"""

standard = """
Andet:
  - standardvarer:
      amount: 1
      unit: plate
"""

menu = "".join(menu_item_template.format(**item) for item in menu_dict["menu_items"])+standard
    
today = datetime.date.today().isocalendar()

#with open("week-{today.week}-{today.year}.yaml", "w") as f:
#    f.write(menu)

menu_d = yaml.load(menu, Loader=yaml.FullLoader)


In [ ]:
tool_call_id = run.required_action.submit_tool_outputs.tool_calls[0]
function_name = run.required_action.submit_tool_outputs.tool_calls[0].function.name
function_arguments = json.loads(run.required_action.submit_tool_outputs.tool_calls[0].function.arguments)
function_name


run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call_id.id,
            "output": "success",
        }
    ],
)

In [ ]:
while run.status != 'completed':
    run = client.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id
    )
    if run.status == 'requires_action':
        break

thread_messages = client.beta.threads.messages.list(thread.id)


In [ ]:
for msg in thread_messages.data[::-1]:
    print(msg.content[0].text.value)


In [14]:
s = write_md_str(menu_d)
save_to_md(s)
md_to_pdf()
open_pdf()
